In [4]:
# imports
import os
os.sys.path.append('../')
from glob import glob
from src.scraping import soup_parser
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_columns', None)

This block iterates through a directory with html files, opening them one by one, formatting them with BeautifulSoup, then running the soup_parser() function to get relevent information into a pandas dataframe. For this project, the `html` directory has around 30,000 files, and the `html2` directory has 20,000.

In [17]:
# instantiate empty list to hold data
data_list = []
# iterate through directory
for f in glob('../html2/tw*.html'):
    with open(f, 'r') as f2:
        # read the html file
        html = f2.read()
        # BeautifulSoups the html
        to_be_parsed = BeautifulSoup(html)
        f2.close()
        # parses the soup
        data = soup_parser(to_be_parsed)
        # adds the data to the list
        data_list.append(data)
# filters the NaN values
data_list = [x for x in data_list if x != None]
# turns the list into a pandas dataframe
df = pd.DataFrame(data_list)

In [21]:
# saves the dataframe as a raw uncleaned dataset
# df.to_csv('../data/parsed_soup_raw_2.csv', index = False)

In [2]:
# df = pd.read_csv('parsed_soup_raw.csv').drop(columns = 'Unnamed: 0')

In [27]:
streamer_df = pd.read_csv('../data/second_random_sample.csv')

In [28]:
streamer_df

,user_id,user_name,game_id,language,viewer_count,game_name,type_left,login,display_name,type_right,broadcaster_type,description,profile_image_url,offline_image_url,view_count,created_at,account_age
0,487481830,skiratadesiire,512709.0,en,3,Call of Duty: Black Ops Cold War,live,skiratadesiire,SkirataDesiire,NaN,affiliate,19 Year Old Call of Duty Streamer ⭐️Stream abo...,https://static-cdn.jtvnw.net/jtv_user_pictures...,NaN,3034.0,2020-01-23 06:16:18.182573+00:00,313 days 08:33:24.315292000
1,449712167,민팡_,509658.0,ko,14,Just Chatting,live,minpang77,민팡_,NaN,affiliate,[곰팡단 대장] 저챗스트리머 & 종겜스트리머,https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,33121.0,2019-07-21 05:51:38.963864+00:00,499 days 08:58:03.534001000
2,462034862,v4ldogod,32982.0,pt,3,Grand Theft Auto V,live,v4ldogod,v4ldogod,NaN,affiliate,NaN,https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,4381.0,2019-09-12 23:32:43.529838+00:00,445 days 15:16:58.968027000
3,63576494,solentka,19357.0,fr,10,Guild Wars 2,live,solentka,Solentka,NaN,affiliate,Ancien rédacteur JV aux goûts éclectiques. En ...,https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,3455.0,2014-06-02 14:29:36.281650+00:00,2374 days 00:20:06.216215000
4,258901661,chickenxop,27471.0,en,1,Minecraft,live,chickenxop,ChickenXop,NaN,affiliate,Me,https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,1003.0,2018-09-15 17:29:49.771773+00:00,807 days 21:19:52.726092000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26674,502963273,roadhog_lover,31376.0,de,5,Terraria,live,roadhog_lover,Roadhog_Lover,NaN,NaN,NaN,https://static-cdn.jtvnw.net/jtv_user_pictures...,NaN,1435.0,2020-03-23 22:57:42.078717+00:00,252 days 15:52:00.419148000
26675,71526179,lleilexe,518184.0,en,7,Phasmophobia,live,lleilexe,lleilEXE,NaN,affiliate,exe!,https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,14726.0,2014-09-19 16:23:08.218548+00:00,2264 days 22:26:34.279317000
26676,161837100,el_cabra,21779.0,es,1,League of Legends,live,el_cabra,el_cabra,NaN,affiliate,NaN,https://static-cdn.jtvnw.net/jtv_user_pictures...,NaN,6631.0,2017-06-28 06:35:55.428649+00:00,1252 days 08:13:47.069216000
26677,147444751,tveehead,518184.0,en,2,Phasmophobia,live,tveehead,TVeeHead,NaN,affiliate,Variety streamer with a heavy slant towards ho...,https://static-cdn.jtvnw.net/jtv_user_pictures...,NaN,1176.0,2017-02-10 20:19:24.956224+00:00,1389 days 18:30:17.541641000


In [29]:
df.login = df.login.map(lambda x: x.split('\n')[1] if '\n' in x else x)
df.login = df.login.str.lower()

In [30]:
working_df = streamer_df.join(df.set_index('login'),
                              on = 'login',
                              lsuffix = '_left',
                              rsuffix = '_right').drop(columns = 'type_right').dropna(axis = 0, subset = {'Hours streamed'})

In [37]:
cleaning_df = working_df.drop(columns = ['user_id',
                                         'user_name',
                                         'viewer_count',
                                         'type_left',
                                         'display_name',
                                         'description',
                                         'created_at',
                                         'profile_image_url',
                                         'offline_image_url',
                                         'game_id',
                                         'Followers',
                                         'Views', 
                                         'Hours watched daily',
                                         'Views to date'
                          ]
               )
cleaning_df.reset_index(drop=True, inplace = True)
cleaning_df.columns = cleaning_df.columns.str.lower().str.replace(' ', '_')
cleaning_df.broadcaster_type = cleaning_df.broadcaster_type.fillna('unpartnered')
cleaning_df.account_age = pd.to_timedelta(cleaning_df.account_age)
cleaning_df.view_count = cleaning_df.view_count.astype('int')
cleaning_list = ['hours_streamed',
             'average_viewers',
             'peak_viewers', 
             'days_of_activity',
             'total_games_streamed',
             'followers_per_stream',
             'views_per_stream',
             'followers_per_hour',
             'views_per_hour',
             'hours_watched',
             'followers_to_date',
             'hours_watched_daily']
cleaning_df[cleaning_list] = cleaning_df[cleaning_list].astype('float')
cleaning_df.average_games = cleaning_df.average_games.map(lambda x: x[:-7]).astype('float')
cleaning_df.active_days_per_week = cleaning_df.active_days_per_week.map(lambda x: x[:-4]).astype('float')
cleaning_df.daily_broadcast_time = cleaning_df.daily_broadcast_time.map(lambda x: x[:-3]).astype('int')
cleaning_df['is_affiliate'] = [0 if x == 'unpartnered' else 1 for x in cleaning_df.broadcaster_type]
cleaning_df['target'] = [1 if x == 'partner' else 0 for x in cleaning_df.broadcaster_type]

In [39]:
cleaning_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17392 entries, 0 to 17391
Data columns (total 23 columns):
language                17392 non-null object
game_name               17230 non-null object
login                   17392 non-null object
broadcaster_type        17392 non-null object
view_count              17392 non-null int64
account_age             17392 non-null timedelta64[ns]
hours_streamed          17392 non-null float64
average_viewers         17392 non-null float64
peak_viewers            17392 non-null float64
days_of_activity        17392 non-null float64
total_games_streamed    17392 non-null float64
daily_broadcast_time    17392 non-null int64
hours_watched_daily     17392 non-null float64
followers_per_stream    17392 non-null float64
views_per_stream        17392 non-null float64
followers_per_hour      17392 non-null float64
views_per_hour          17392 non-null float64
hours_watched           17392 non-null float64
followers_to_date       17392 non-null float6

In [41]:
cleaning_df.to_csv('../data/streamer_data_2.csv', index = False)